In [1]:
# Import aller benötigten Programmbibliotheken

import pandas as pd
import numpy as np
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
import random
import implicit
import nmslib
import annoy
from implicit.evaluation import precision_at_k, mean_average_precision_at_k

# Import des Online Retail-Datensatzes

OnlineRetail = pd.read_excel(r"C:\Users\patri\Google Drive\Uni!\Master WiInfo\IV. Semester\Seminararbeit\Datensätze\Online Retail\Online Retail.xlsx")

# Anzeigen des 'Datensatzkopfes'

OnlineRetail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [2]:
# Der Datensatz enhält insgesamt 541909 Zeilen

OnlineRetail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
InvoiceNo      541909 non-null object
StockCode      541909 non-null object
Description    540455 non-null object
Quantity       541909 non-null int64
InvoiceDate    541909 non-null datetime64[ns]
UnitPrice      541909 non-null float64
CustomerID     406829 non-null float64
Country        541909 non-null object
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [3]:
# Zeilen, die keine CustomerID enthalten, entfernen. 
# Ohne CustomerID kann später in der Matrix keine Zuordnung zu einem Produkt erfolgen. 

CleanedData = OnlineRetail.loc[pd.isnull(OnlineRetail.CustomerID) == False]

In [4]:
# Nach dem Entfernen der Zeilen ohne CustomerID enthält der Datensatz noch 406829 Zeilen

CleanedData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 406829 entries, 0 to 541908
Data columns (total 8 columns):
InvoiceNo      406829 non-null object
StockCode      406829 non-null object
Description    406829 non-null object
Quantity       406829 non-null int64
InvoiceDate    406829 non-null datetime64[ns]
UnitPrice      406829 non-null float64
CustomerID     406829 non-null float64
Country        406829 non-null object
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 27.9+ MB


In [5]:
#Erstellen einer separaten Tabelle, die nur den StockCode und die eine Description der Artikel enthält

ItemData = CleanedData[["StockCode", "Description"]].drop_duplicates()

# Formatieren der Spalte "StockCode" von einem numerischen Wert in einen String

ItemData["StockCode"] = ItemData["StockCode"].astype(str)

In [6]:
# Die separate Tabelle 'ItemData' enthält die 3916 Produkte des Unternehmens
 
ItemData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3916 entries, 0 to 540421
Data columns (total 2 columns):
StockCode      3916 non-null object
Description    3916 non-null object
dtypes: object(2)
memory usage: 91.8+ KB


In [7]:
# Anzeigen des 'Tabellenkopfes'

ItemData.head()

,StockCode,Description
0,85123A,WHITE HANGING HEART T-LIGHT HOLDER
1,71053,WHITE METAL LANTERN
2,84406B,CREAM CUPID HEARTS COAT HANGER
3,84029G,KNITTED UNION FLAG HOT WATER BOTTLE
4,84029E,RED WOOLLY HOTTIE WHITE HEART.


In [8]:
# Der Datensatz enthält nicht nur Verkäufe, sondern auch Rückgaben, da negative Mengen in der Spalte 'Quantity' vorhanden sind.

CleanedData["Quantity"].describe()

count    406829.000000
mean         12.061303
std         248.693370
min      -80995.000000
25%           2.000000
50%           5.000000
75%          12.000000
max       80995.000000
Name: Quantity, dtype: float64

In [9]:
# Überführen der CustomerID in numerischen Wert (int)
CleanedData["CustomerID"] = CleanedData["CustomerID"].astype(int)
CleanedData.info()

C:\Users\patri\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


<class 'pandas.core.frame.DataFrame'>
Int64Index: 406829 entries, 0 to 541908
Data columns (total 8 columns):
InvoiceNo      406829 non-null object
StockCode      406829 non-null object
Description    406829 non-null object
Quantity       406829 non-null int64
InvoiceDate    406829 non-null datetime64[ns]
UnitPrice      406829 non-null float64
CustomerID     406829 non-null int32
Country        406829 non-null object
dtypes: datetime64[ns](1), float64(1), int32(1), int64(1), object(4)
memory usage: 26.4+ MB


In [10]:
# Reduzieren des Online Retail Datensatzes auf die benötigten Daten für die Kunden-Produkt-Matrix:

CleanedData = CleanedData[["StockCode", "Quantity", "CustomerID"]]

# Gruppieren und aufsummieren der Daten, um Rückgaben und Käufe 'auszugleichen':

GroupedData = CleanedData.groupby(["CustomerID", "StockCode"]).sum().reset_index()

# Sollten durch das Aufsummieren von Rückgaben und Käufen 0-Werte entstanden sein, werden diese nun durch 1 ersetzt.
# Dies gibt weiterhin ein Kaufinteresse des Kunden an diesem Artikel wieder.

GroupedData["Quantity"].loc[GroupedData["Quantity"] == 0] = 1

# Anzeigen des 'Tabellenkopfes' des reduzierten Datensatzes

GroupedData.head()

C:\Users\patri\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,CustomerID,StockCode,Quantity
0,12346,23166,1
1,12347,16008,24
2,12347,17021,36
3,12347,20665,6
4,12347,20719,40


In [11]:
# Es sind weiterhin mehrere Werte, die kleiner 0 sind,im Datensatz vorhanden d.h. im betrachteten Zeitraum wurden mehr Produkte zurückgegeben als gekauft wurden.

GroupedData["Quantity"].describe()

count    267615.000000
mean         18.341240
std          97.388138
min       -9360.000000
25%           2.000000
50%           6.000000
75%          12.000000
max       12540.000000
Name: Quantity, dtype: float64

In [12]:
# Mengen, die weiterhin kleiner 0 sind, werden herausgefiltert (d.h. mehr Rückgaben als Verkäufe).
# Dies wird getan, da sich diese negativen Mengen nicht korrekt gegenüber früheren Bestelleungen gegengerechnen lassen.

PurchasedData = GroupedData[GroupedData["Quantity"] > 0]
PurchasedData.head()

,CustomerID,StockCode,Quantity
0,12346,23166,1
1,12347,16008,24
2,12347,17021,36
3,12347,20665,6
4,12347,20719,40


In [13]:
# Erstellen einer Liste von Kunden aus dem DataFrame 'PurchasedData'

Customers = list(np.sort(PurchasedData["CustomerID"].unique()))

# Erstellen einer Liste von Produkten aus dem DataFrame 'PurchasedData'

Products = list(PurchasedData["StockCode"].unique())

# Erstellen einer Liste von Mengen aus dem DataFrame 'PurchasedData'

Quantities = list(PurchasedData["Quantity"])

In [14]:
# Implicit benötigt eine Sparse Matrix der Form Produkt / Kunde / Bewertung

# Erstellen der Spalten der Matrix

cols = PurchasedData["CustomerID"].astype("category", categories = Customers).cat.codes

# Erstellen der Reihen der Matrix

rows = PurchasedData["StockCode"].astype("category", categories = Products).cat.codes

# Erstellen der Matrix

PurchaseMatrix = sparse.csr_matrix((Quantities, (rows, cols)), shape = (len(Products), len(Customers)))

C:\Users\patri\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3296: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  exec(code_obj, self.user_global_ns, self.user_ns)


In [15]:
# Anzeigen der Matrix

PurchaseMatrix

<3664x4338 sparse matrix of type '<class 'numpy.int32'>'
	with 266723 stored elements in Compressed Sparse Row format>

In [16]:
# Berechnung der 'Sparsity' der Matrix
# Mit einer Sparsity von 98,32 % ist die Matrix ausreichend befüllt, um aussagelräftige Empfehlungen generieren zu können. 

MatrixSize = PurchaseMatrix.shape[0]*PurchaseMatrix.shape[1]
NumberPurchases = len(PurchaseMatrix.nonzero()[0])
Sparsity = 100*(1 - (NumberPurchases/MatrixSize))
Sparsity

98.32190920694744

In [17]:
# Für das Training und die spätere Bewertung der Algorithmen wird der Datensatz in Trainings- & Testdaten unterteilt.
# 80 % der Daten werden für das Training der Algorithmen verwendet.

def train(ratings, pct_test = 0.2):
    test_set = ratings.copy()
    test_set[test_set != 0] = 1
    training_set = ratings.copy()
    nonzero_inds = training_set.nonzero()
    nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1]))
    random.seed(0)
    num_samples = int(np.ceil(pct_test * len(nonzero_pairs)))
    samples = random.sample(nonzero_pairs, num_samples)
    user_inds = [index[0] for index in samples]
    item_inds = [index[1] for index in samples]
    training_set[user_inds, item_inds] = 0
    training_set.eliminate_zeros()
    return training_set, test_set, list(set(user_inds))

ProductTrain, ProductTest, ProductUsersAltered = train(PurchaseMatrix, pct_test = 0.2)

In [18]:
# Anzeigend er Trainings-Matrix

ProductTrain

<3664x4338 sparse matrix of type '<class 'numpy.int32'>'
	with 213378 stored elements in Compressed Sparse Row format>

In [19]:
# Anzeigen der Trainings-Matrix

ProductTest

<3664x4338 sparse matrix of type '<class 'numpy.int32'>'
	with 266723 stored elements in Compressed Sparse Row format>

In [28]:
# Initialisieren der verschiedenen Algorithmen

# Alternating Least Square

ALS = implicit.als.AlternatingLeastSquares(iterations = 84, factors = 243, regularization = 0.68)

# Bayesian Personalized Ranking

BPR = implicit.bpr.BayesianPersonalizedRanking(iterations = 78, factors = 191, regularization = 0, learning_rate = 0.01 )

# Alternating Least Squares unter Verwendung der NMSLib zur Berechnung der Nearest Neigbours

NMSL_ALS = implicit.approximate_als.NMSLibAlternatingLeastSquares()

# Alternating Least Squares unter Verwendung von Annoy zur Berechnung ähnlicher Produkte 

AN_ALS = implicit.approximate_als.AnnoyAlternatingLeastSquares(n_trees = 373, search_k = -60)

In [29]:
# Trainieren der verschiedenen Modelle

# Alternating Least Square

ALS.fit(ProductTrain)

# Bayesian Personalized Ranking

BPR.fit(ProductTrain)

# Alternating Least Squares unter Verwendung der NMSLib zur Berechnung der Nearest Neigbours

NMSL_ALS.fit(ProductTrain)

# Alternating Least Squares unter Verwendung von Annoy zur Berechnung ähnlicher Produkte 

AN_ALS.fit(ProductTrain)

100%|██████████| 84.0/84 [00:12<00:00,  6.59it/s]
100%|██████████| 78/78 [00:03<00:00, 21.25it/s, correct=93.01%, skipped=9.07%]
100%|██████████| 15.0/15 [00:00<00:00, 17.55it/s]
100%|██████████| 15.0/15 [00:01<00:00, 13.82it/s]


In [30]:
UserItems = ProductTrain.T.tocsr()
CustomersArr = np.array(Customers)
ProductsArr = np.array(Products)

In [31]:
# Definieren der Empfehlungsfunktion mit der Auswahlmöglichkeit des Algorithmus

def recommendProduct(rec_type, customer_id, mf_train, customer_list, item_list, num_items,item_database):
    cust_ind = np.where(customer_list == customer_id)[0][0]
    
    counter = 0
    FirstLevelRecommendations = []
    for i in range(0, num_items - 1):
        RecSysResults = rec_type.recommend(cust_ind, UserItems)[counter]
        counter = counter + 1
        RecSysResults = RecSysResults[0]
        FirstLevelRecommendations.append(RecSysResults)
    RecommendationsList = []
    for index in FirstLevelRecommendations:
        ProductCode = ProductsArr[index]
        RecommendationsList.append(ProductCode)
    return item_database[item_database["StockCode"].isin (RecommendationsList)]

In [41]:
recommendProduct(BPR, 17548, ProductTrain, CustomersArr, ProductsArr, 10, ItemData)

,StockCode,Description
9,84879,ASSORTED COLOUR BIRD ORNAMENT
172,85049A,TRADITIONAL CHRISTMAS RIBBONS
325,85049E,SCANDINAVIAN REDS RIBBONS
476,84347,ROTATING SILVER ANGELS T-LIGHT HLDR
562,22077,6 RIBBONS RUSTIC CHARM
991,22904,CALENDAR PAPER CUT DESIGN
1133,22584,PACK OF 6 PANNETONE GIFT BOXES
5307,22078,RIBBON REEL LACE DESIGN
6673,22080,RIBBON REEL POLKADOTS
476984,22584,PACK OF 6 PANETTONE GIFT BOXES


In [39]:
recommendProduct(ALS, 17548, ProductTrain, CustomersArr, ProductsArr, 10, ItemData)

,StockCode,Description
9,84879,ASSORTED COLOUR BIRD ORNAMENT
478,22736,RIBBON REEL MAKING SNOWMEN
562,22077,6 RIBBONS RUSTIC CHARM
695,22113,GREY HEART HOT WATER BOTTLE
773,22925,BLUE GIANT GARDEN THERMOMETER
1133,22584,PACK OF 6 PANNETONE GIFT BOXES
1299,22567,20 DOLLY PEGS RETROSPOT
5307,22078,RIBBON REEL LACE DESIGN
6673,22080,RIBBON REEL POLKADOTS
476984,22584,PACK OF 6 PANETTONE GIFT BOXES


In [40]:
recommendProduct(NMSL_ALS, 17548, ProductTrain, CustomersArr, ProductsArr, 10, ItemData)

,StockCode,Description
9,84879,ASSORTED COLOUR BIRD ORNAMENT
325,85049E,SCANDINAVIAN REDS RIBBONS
404,84378,SET OF 3 HEART COOKIE CUTTERS
562,22077,6 RIBBONS RUSTIC CHARM
626,22738,RIBBON REEL SNOWY VILLAGE
1026,22907,PACK OF 20 NAPKINS PANTRY DESIGN
1299,22567,20 DOLLY PEGS RETROSPOT
5307,22078,RIBBON REEL LACE DESIGN
6673,22080,RIBBON REEL POLKADOTS


In [42]:
recommendProduct(AN_ALS, 17548, ProductTrain, CustomersArr, ProductsArr, 10, ItemData)

,StockCode,Description
9,84879,ASSORTED COLOUR BIRD ORNAMENT
325,85049E,SCANDINAVIAN REDS RIBBONS
399,85049G,CHOCOLATE BOX RIBBONS
562,22077,6 RIBBONS RUSTIC CHARM
793,22737,RIBBON REEL CHRISTMAS PRESENT
1299,22567,20 DOLLY PEGS RETROSPOT
5307,22078,RIBBON REEL LACE DESIGN
6673,22080,RIBBON REEL POLKADOTS
8523,84978,HANGING HEART JAR T-LIGHT HOLDER
